自定义层
首先，我们构造一个没有任何参数的自定义层。 
下面的CenteredLayer类要从其输入中减去均值。要构建它，我们只需继承基础层类并实现前向传播功能。

In [4]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
#让我们向该层提供一些数据，验证它是否能按预期工作。
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [6]:
#现在我们将层作为组件合并到更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为额外的健全性检查，我们可以在向该网络发送随机数据后，检查均值是否为0。 由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。

In [7]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

带参数的层
让我们实现自定义版本的全连接层。 回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。 在此实现中，我们使用修正线性单元作为激活函数。 该层需要输入参数：in_units和units，分别表示输入和输出的数量。

In [9]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [10]:
#接下来，我们实例化MyLinear类并访问其模型参数。
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.8345, -0.0892, -1.1785],
        [-1.2719,  0.7025,  1.6422],
        [-0.5741,  2.9101, -0.3098],
        [ 0.3991, -1.4259,  1.0113],
        [-0.2224,  1.2219, -0.2173]], requires_grad=True)

In [11]:
#我们可以使用自定义层直接执行前向传播计算。
linear(torch.rand(2, 5))

tensor([[0.0000, 1.9793, 0.3683],
        [0.4639, 0.9721, 0.6073]])

In [12]:
#我们还可以使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层。
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[14.8026],
        [ 4.5851]])

读写文件
学习如何加载和存储权重向量和整个模型了


In [14]:
#对于单个张量，我们可以直接调用load和save函数分别读写它们。 
#这两个函数都要求我们提供一个名称，save要求将要保存的变量作为输入。
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [15]:
#我们可以存储一个张量列表，然后把它们读回内存。
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [17]:
#我们甚至可以写入或读取从字符串映射到张量的字典。当我们要读取或写入模型中的所有权重时，这很方便。
mydict = {'x':x, 'y':y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数
深度学习框架提供了内置函数来保存和加载整个网络
需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。例如，如果我们有一个3层多层感知机，我们需要单独指定架构。因为模型本身可以包含任意代码，所以模型本身难以序列化。因此，为了恢复模型，我们需要用代码生成架构， 然后从磁盘加载参数。让我们从熟悉的多层感知机开始尝试一下。

In [20]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2,20))
Y = net(X)
        

接下来，我们将模型的参数存储在一个叫做“mlp.params”的文件中。

In [21]:
torch.save(net.state_dict(), 'MLP.params')

为了恢复模型，我们实例化了原始多层感知机模型的一个备份。 这里我们不需要随机初始化模型参数，而是直接读取文件中存储的参数。

In [22]:
clone = MLP()
clone.load_state_dict(torch.load('MLP.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的X时， 两个实例的计算结果应该相同。 让我们来验证一下。

In [23]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])